In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
import pandas as pd

In [6]:
tables = []
for i in range(25):
    url = f"https://mathstats.uncg.edu/sites/pauli/congruence/csg{i}.html"
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table')
    tables.append(table)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [7]:
headers = "Title, Name, Index, con, len, c_2, c_3, Cusps, Gal, Supergroups, Subgroups".split(', ')
data = []

for genus, table in enumerate(tables):
    print(genus)
    for row in table.find_all('tr'):
        if "level" in row.get('id'):
            continue
        else:
            cols_th = row.find_all('th')
            cols_td = row.find_all('td')
            cols = cols_th + cols_td
            cols = [ele.decode_contents().strip() for ele in cols if ele]
            
            # For subgroups
            links = cols_td[-1].find_all('a')
            links = [link.decode_contents() for link in links if link]
            cols[-1] = links

            # For supergroups
            links = cols_td[-2].find_all('a')
            links = [link.decode_contents() for link in links if link]
            cols[-2] = links
            
            # insert sup in the first td
            cols.pop(1), cols.pop(7)

            row = dict(zip(headers, cols))
            data.append(row)

df = pd.DataFrame(data)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [8]:
df

,Title,Name,Index,con,len,c_2,c_3,Cusps,Gal,Supergroups,Subgroups
0,1A<sup>0</sup>,Γ,1,1,1,1,1,1<sup>1</sup>,1<sup>1</sup>,[],"[2A<sup>0</sup>, 2B<sup>0</sup>, 3A<sup>0</sup..."
1,2A<sup>0</sup>,Γ<sup>2</sup>,2,1,1,0,2,2<sup>1</sup>,1<sup>1</sup>,[1A<sup>0</sup>],"[2C<sup>0</sup>, 4D<sup>0</sup>, 6A<sup>0</sup..."
2,2B<sup>0</sup>,"Γ<sub>0</sub>(2),\nΓ<sub>1</sub>(2)",3,1,3,1,0,1<sup>1</sup>\n2<sup>1</sup>,1<sup>3</sup>,[1A<sup>0</sup>],"[2C<sup>0</sup>, 4B<sup>0</sup>, 4C<sup>0</sup..."
3,2C<sup>0</sup>,Γ(2),6,1,1,0,0,2<sup>3</sup>,1<sup>1</sup>,"[2A<sup>0</sup>, 2B<sup>0</sup>]","[4E<sup>0</sup>, 6I<sup>0</sup>, 6C<sup>1</sup..."
4,3A<sup>0</sup>,Γ<sup>3</sup>,3,1,1,3,0,3<sup>1</sup>,1<sup>1</sup>,[1A<sup>0</sup>],"[3C<sup>0</sup>, 6B<sup>0</sup>, 6D<sup>0</sup..."
...,...,...,...,...,...,...,...,...,...,...,...
8104,336E<sup>24</sup>,,336,2,168,18,0,336<sup>1</sup>,16<sup>2</sup>\n32<sup>2</sup>\n48<sup>4</sup>...,[168C<sup>10</sup>],[]
8105,336F<sup>24</sup>,,336,2,168,18,0,336<sup>1</sup>,16<sup>2</sup>\n32<sup>2</sup>\n48<sup>4</sup>...,[168D<sup>10</sup>],[]
8106,336G<sup>24</sup>,,336,2,168,18,0,336<sup>1</sup>,16<sup>2</sup>\n32<sup>2</sup>\n48<sup>4</sup>...,"[48B<sup>3</sup>, 168C<sup>10</sup>]",[]
8107,336H<sup>24</sup>,,336,2,168,18,0,336<sup>1</sup>,16<sup>2</sup>\n32<sup>2</sup>\n48<sup>4</sup>...,"[48B<sup>3</sup>, 168D<sup>10</sup>]",[]


In [10]:
def get_nested_subgroup(title, visited=None):
    # if visited is None:
    #     print(title)
    if visited is None:
        visited = set()
    # get row of title
    if title in visited:
        return set()
    visited.add(title)
    row = df[df['Title'] == title]
    subgroups = set(row['Subgroups'].values[0])
    nested_subgroups = set()
    nested_subgroups |= set(row['Subgroups'].values[0])
    for subgroup in subgroups:
        nested_subgroups |= get_nested_subgroup(subgroup, visited)
    return nested_subgroups

def get_nested_supergroup(title, visited=None):
    # if visited is None:
    #     print(title)
    if visited is None:
        visited = set()
    # get row of title
    if title in visited:
        return set()
    visited.add(title)
    row = df[df['Title'] == title]
    supergroups = set(row['Supergroups'].values[0])
    nested_supergroups = set()
    nested_supergroups |= set(row['Supergroups'].values[0])
    for supergroup in supergroups:
        nested_supergroups |= get_nested_supergroup(supergroup, visited)
    return nested_supergroups

In [11]:
df['Nested_supergroups'] = df['Title'].apply(get_nested_supergroup)
df['Nested_Subgroups'] = df['Title'].apply(get_nested_subgroup)

In [12]:
df.head()

,Title,Name,Index,con,len,c_2,c_3,Cusps,Gal,Supergroups,Subgroups,Nested_supergroups,Nested_Subgroups
0,1A<sup>0</sup>,Γ,1,1,1,1,1,1<sup>1</sup>,1<sup>1</sup>,[],"[2A<sup>0</sup>, 2B<sup>0</sup>, 3A<sup>0</sup...",{},"{36K<sup>4</sup>, 24E<sup>1</sup>, 45A<sup>20<..."
1,2A<sup>0</sup>,Γ<sup>2</sup>,2,1,1,0,2,2<sup>1</sup>,1<sup>1</sup>,[1A<sup>0</sup>],"[2C<sup>0</sup>, 4D<sup>0</sup>, 6A<sup>0</sup...",{1A<sup>0</sup>},"{90K<sup>22</sup>, 198B<sup>22</sup>, 16A<sup>..."
2,2B<sup>0</sup>,"Γ<sub>0</sub>(2),\nΓ<sub>1</sub>(2)",3,1,3,1,0,1<sup>1</sup>\n2<sup>1</sup>,1<sup>3</sup>,[1A<sup>0</sup>],"[2C<sup>0</sup>, 4B<sup>0</sup>, 4C<sup>0</sup...",{1A<sup>0</sup>},"{24E<sup>1</sup>, 16F<sup>5</sup>, 16A<sup>11<..."
3,2C<sup>0</sup>,Γ(2),6,1,1,0,0,2<sup>3</sup>,1<sup>1</sup>,"[2A<sup>0</sup>, 2B<sup>0</sup>]","[4E<sup>0</sup>, 6I<sup>0</sup>, 6C<sup>1</sup...","{2A<sup>0</sup>, 1A<sup>0</sup>, 2B<sup>0</sup>}","{40R<sup>13</sup>, 16A<sup>11</sup>, 96A<sup>1..."
4,3A<sup>0</sup>,Γ<sup>3</sup>,3,1,1,3,0,3<sup>1</sup>,1<sup>1</sup>,[1A<sup>0</sup>],"[3C<sup>0</sup>, 6B<sup>0</sup>, 6D<sup>0</sup...",{1A<sup>0</sup>},"{36K<sup>4</sup>, 246A<sup>19</sup>, 45A<sup>2..."


In [38]:
import re
import pandas as pd
import openpyxl
from openpyxl.utils import get_column_letter

# Creating a mapping for superscript conversion
SUPERSCRIPT_MAPPING = {
    "0": "⁰",
    "1": "¹",
    "2": "²",
    "3": "³",
    "4": "⁴",
    "5": "⁵",
    "6": "⁶",
    "7": "⁷",
    "8": "⁸",
    "9": "⁹"
}
SUBSCRIPT_MAPPING = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉"
}

def replace_with_subscript(value):
    if isinstance(value, str):
        pattern = re.compile(r'<sub>(\d+)</sub>')

        def repl(match):
            return ''.join(SUBSCRIPT_MAPPING[digit] for digit in match.group(1))
        value = pattern.sub(repl, value)
    return value

def replace_with_superscript(value):
    if isinstance(value, str):
        pattern = re.compile(r'<sup>(\d+)</sup>')

        def repl(match):
            return ''.join(SUPERSCRIPT_MAPPING[digit] for digit in match.group(1))
        value = pattern.sub(repl, value)
    return value
def stringify_lists(value):
    if isinstance(value, (set, list)):
        value = ", ".join(str(i) for i in value)
        
    return value

In [39]:
df[['Supergroups', 'Subgroups', 'Nested_supergroups', 'Nested_Subgroups']] = df[['Supergroups', 'Subgroups', 'Nested_supergroups', 'Nested_Subgroups']].applymap(stringify_lists)
# df = df.applymap(replace_with_superscript)
# df = df.applymap(replace_with_superscript)

In [40]:
'Γ₀(2),\nΓ₁(2)	'.replace('\n',', ')

'Γ₀(2),, Γ₁(2)\t'

In [41]:
df.head()

,Title,Name,Index,con,len,c_2,c_3,Cusps,Gal,Supergroups,Subgroups,Nested_supergroups,Nested_Subgroups
0,1A⁰,Γ,1,1,1,1,1,1¹,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,2A⁰,Γ²,2,1,1,0,2,2¹,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,2B⁰,"Γ₀(2),\nΓ₁(2)",3,1,3,1,0,1¹\n2¹,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,2C⁰,Γ(2),6,1,1,0,0,2³,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,3A⁰,Γ³,3,1,1,3,0,3¹,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [97]:
df['Cusps'] = df['Cusps'].str.replace('\n', ', ')
df['Cusps'] = df['Cusps'].str.replace(',, ', ', ')
df['Cusps'] = df['Cusps'].str.replace(',,', ', ')

df['Gal'] = df['Gal'].str.replace('\n', ', ')
df['Gal'] = df['Gal'].str.replace(',, ', ', ')
df['Gal'] = df['Gal'].str.replace(',,', ', ')

df['Name'] = df['Name'].str.replace('\n', ', ')
df['Name'] = df['Name'].str.replace(',, ', ', ')
df['Name'] = df['Name'].str.replace(',,', ', ')


In [98]:
df.head()

,Title,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_supergroups,Nested_Subgroups
0,1A⁰,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,2A⁰,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,2B⁰,"Γ₀(2), Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,2C⁰,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,3A⁰,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [94]:
SUPERSCRIPT_TO_DIGIT_MAPPING = {
    "⁰": "0", "¹": "1", "²": "2", "³": "3", "⁴": "4",
    "⁵": "5", "⁶": "6", "⁷": "7", "⁸": "8", "⁹": "9"
}

def sum_of_superscripts(input_str):
    pattern = re.compile(r'[⁰¹²³⁴⁵⁶⁷⁸⁹]+')
    superscripts = pattern.findall(input_str)
    numbers = [int(''.join(SUPERSCRIPT_TO_DIGIT_MAPPING[sup] for sup in superscript)) for superscript in superscripts]
    return sum(numbers)

In [95]:
df['Sum Of Cusps'] = df['Cusps'].apply(sum_of_superscripts)
# Get a list of all column names
cols = df.columns.tolist()

# Find the current position of 'Cusps' and 'Sum Of Cusps'
cusps_index = cols.index('Cusps')
sum_of_cusps_index = cols.index('Sum Of Cusps')

# Remove 'Sum Of Cusps' from its current position
cols.pop(sum_of_cusps_index)

# Insert 'Sum Of Cusps' right after 'Cusps'
cols.insert(cusps_index + 1, 'Sum Of Cusps')

# Reorder the DataFrame
df = df[cols]


In [96]:
df.head()

,Title,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_supergroups,Nested_Subgroups
0,1A⁰,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,2A⁰,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,2B⁰,"Γ₀(2),, Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,2C⁰,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,3A⁰,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [158]:
def generate_links(value):
    # Regular expression pattern to match group names and superscripts
    pattern = re.compile(r'([a-zA-Z\d]+)([⁰¹²³⁴⁵⁶⁷⁸⁹]*)')
    
    # Split the string into groups
    groups = value.split(', ')
    
    # Process each group
    links = []
    for group in groups:
        # Extract the name and superscript
        match = pattern.match(group)
        if match:
            name, superscript = match.groups()
            # Convert the superscript to an integer
            superscript = ''.join(SUPERSCRIPT_TO_DIGIT_MAPPING[digit] for digit in superscript)
            # Generate the link using Excel HYPERLINK function
            url = f'https://mathstats.uncg.edu/sites/pauli/congruence/csg{superscript}.html#group{name}{superscript}'
            link = f'HYPERLINK("{url}", "{group}")'
            links.append(link)
    
    # Join the links into a single string
    if links:
        return '='+'&CHAR(44)&CHAR(32)&'.join(links)
    else:
        return''


In [152]:
for c in ", ":print(ord(c))

44
32


In [165]:
df = df_copy.copy()

In [154]:
df = df.rename(columns={'Nested_supergroups': 'Nested_Supergroups'})

In [166]:
df['Genus'] = df['Title'].apply(sum_of_superscripts)
# Get a list of all column names
cols = df.columns.tolist()

# Find the current position of 'Cusps' and 'Sum Of Cusps'
cusps_index = cols.index('Title')
sum_of_cusps_index = cols.index('Genus')

# Remove 'Sum Of Cusps' from its current position
cols.pop(sum_of_cusps_index)

# Insert 'Sum Of Cusps' right after 'Cusps'
cols.insert(cusps_index + 1, 'Genus')

# Reorder the DataFrame
df = df[cols]


In [167]:
df_copy = df.copy(deep=True)
df_copy.head()

,Title,Genus,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_Supergroups,Nested_Subgroups
0,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,"Γ₀(2), Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [161]:
# df['Subgroups'] = df['Subgroups'].apply(generate_links)
# df['Nested_Subgroups'] = df['Nested_Subgroups'].apply(generate_links)

# df['Supergroups'] = df['Supergroups'].apply(generate_links)
# df['Nested_Supergroups'] = df['Nested_Supergroups'].apply(generate_links)
df['Title'] = df['Title'].apply(generate_links)

In [168]:
df.head()

,Title,Genus,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_Supergroups,Nested_Subgroups
0,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,"Γ₀(2), Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [177]:
df_copy2 = df.copy()
df_copy2.head()

,Title,Genus,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_Supergroups,Nested_Subgroups
0,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"36K⁴, 24E¹, 45A²⁰, 16F⁵, 198B²², 24L¹¹, 114A²²..."
1,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"90K²², 198B²², 16A¹¹, 114A²², 42F⁷, 140C²³, 14..."
2,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,"Γ₀(2), Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16F⁵, 16A¹¹, 36L⁸, 90D²⁰, 24M¹⁹, 72F¹⁴, ..."
3,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E², 10A³, 14B³, 2...","2A⁰, 1A⁰, 2B⁰","40R¹³, 16A¹¹, 96A¹⁸, 12I⁰, 40X¹⁷, 84K²², 12G³,..."
4,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"36K⁴, 246A¹⁹, 45A²⁰, 24E¹, 24L¹¹, 15A¹⁹, 36L⁸,..."


In [178]:
def remove_elements_with_large_superscripts(input_str):
    elements = input_str.split(', ')
    filtered_elements = []
    for element in elements:
        superscript_value = sum_of_superscripts(element)
        if superscript_value <= 2:
            filtered_elements.append(element)
    return ', '.join(filtered_elements)

df['Subgroups'] = df['Subgroups'].apply(remove_elements_with_large_superscripts)
df['Nested_Subgroups'] = df['Nested_Subgroups'].apply(remove_elements_with_large_superscripts)
df['Supergroups'] = df['Supergroups'].apply(remove_elements_with_large_superscripts)
df['Nested_Supergroups'] = df['Nested_Supergroups'].apply(remove_elements_with_large_superscripts)

In [180]:
df = df[df['Genus'] <= 2]

In [181]:
df.head()

,Title,Genus,Name,Index,con,len,c_2,c_3,Cusps,Sum Of Cusps,Gal,Supergroups,Subgroups,Nested_Supergroups,Nested_Subgroups
0,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ,1,1,1,1,1,1¹,1,1¹,,"2A⁰, 2B⁰, 3A⁰, 3B⁰, 4A⁰, 5A⁰, 5B⁰, 5C⁰, 7A⁰, 7...",,"24E¹, 14D², 48A⁰, 20G¹, 13B⁰, 11A⁰, 8J¹, 8B², ..."
1,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ²,2,1,1,0,2,2¹,1,1¹,1A⁰,"2C⁰, 4D⁰, 6A⁰, 6C⁰, 10A⁰, 14B⁰, 6A¹, 10A¹, 10C...",1A⁰,"14D², 16F⁰, 18A², 16G⁰, 6C¹, 8J¹, 8E⁰, 12P¹, 3..."
2,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,"Γ₀(2), Γ₁(2)",3,1,3,1,0,"1¹, 2¹",2,1³,1A⁰,"2C⁰, 4B⁰, 4C⁰, 6D⁰, 6F⁰, 10C⁰, 10B¹, 10F¹, 14B...",1A⁰,"24E¹, 16B⁰, 48A⁰, 32E¹, 8B², 4F⁰, 16H⁰, 18P², ..."
3,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ(2),6,1,1,0,0,2³,3,1¹,"2A⁰, 2B⁰","4E⁰, 6I⁰, 6C¹, 10G¹, 10B², 14E²","2A⁰, 1A⁰, 2B⁰","12I⁰, 18J¹, 4G⁰, 12F², 16G⁰, 6C¹, 12B², 12P¹, ..."
4,"=HYPERLINK(""https://mathstats.uncg.edu/sites/p...",0,Γ³,3,1,1,3,0,3¹,1,1¹,1A⁰,"3C⁰, 6B⁰, 6D⁰, 9A⁰, 12A⁰, 15B⁰, 21A⁰, 6A¹, 15A...",1A⁰,"24E¹, 48A⁰, 18K², 3C⁰, 18P², 12N¹, 9A², 12U¹, ..."


In [182]:
output_file = 'math_groups_output_9.xlsx'
# Save the DataFrame to an Excel file
df.to_excel(output_file, index=False) 